In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [ ]:
! kaggle datasets download jessicali9530/celeba-dataset

celeba-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets download jessicali9530/celeba-dataset

celeba-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip "celeba-dataset.zip" -d "/celeba/"

Archive:  celeba-dataset.zip
replace /celeba/img_align_celeba/img_align_celeba/000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /celeba/img_align_celeba/img_align_celeba/000001.jpg  
replace /celeba/img_align_celeba/img_align_celeba/000002.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /celeba/img_align_celeba/img_align_celeba/000002.jpg  
replace /celeba/img_align_celeba/img_align_celeba/000003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /celeba/img_align_celeba/img_align_celeba/000004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#dataroot = "/celeba"
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import cv2 
import glob

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
pip install pretrainedmodels

     |████████████████████████████████| 58 kB 3.9 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=983c59a4557a0c55cd7ffdd3a2cecb12141e7b91b715253163b0064fb30835cf
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [ ]:
import pretrainedmodels
model_name = 'xception'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


  0%|          | 0.00/87.4M [00:00<?, ?B/s]

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        #self.model = Xception(num_classes = 1000)
        #model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
        #self.checkpoint = torch.load("/content/xception-43020ad28.pth")
        #self.xception.load_state_dict(self.checkpoint)
        self.model = model
        #for param in self.model.parameters():
         #  param.requires_grad = False
        #self.fc_inputs = self.xception.fc.in_features
        self.model.last_linear = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
        
    def forward(self, input):
        return self.model(input)

In [ ]:
batch_size = 32
momentum = 0.9
# lr = 0.001
epochs = 10
log_interval = 10

count1 = 0
count2 = 0

X = []
y = []

# for i in glob.glob('fake_face_dcgan/*'):
#   count1+=1 
#   im = cv2.imread(i)
#   X.append(im)
#   y.append(0)
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)
model_G = Generator(ngpu)#.to(device)
model_G.load_state_dict(torch.load('/content/drive/MyDrive/CV_Proj/DCGAN_generator.pth', map_location=torch.device('cpu'))) #.to(device)

for i in range(50):
  count1 += 64 
  fixed_noise = torch.randn(64, nz, 1, 1, device=device)
  with torch.no_grad():
    fake = model_G(fixed_noise).detach().cpu()
  # print(fake.shape)
  # print("BEFORE", torch.max(fake), torch.min(fake))
  
  # fake = (fake -0.5)/0.5
  # print("AFTER", torch.max(fake), torch.min(fake))
  X.append(fake)
  y.append(torch.zeros(len(fake)))

X =torch.cat(X, 0)
y = torch.cat(y,0)
print(X.shape, y.shape)



cpu
torch.Size([3200, 3, 64, 64]) torch.Size([3200])


In [ ]:
xception

NameError: ignored

In [ ]:
count2 = 0
import numpy as np 
from PIL import Image

transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
for i in glob.glob(dataroot + '/img_align_celeba/img_align_celeba/*'):
  count2+=1 
  if (count2<=3200):
    im = Image.open(i)
    # im = cv2.imread(i)
    im = transform(im)

    # print(torch.max(im), torch.min(im))
    # print(im.shape)
    # print(im.shape)
    # print("BEFORE ANYTHING", np.max(im), np.min(im))
    # im = cv2.resize(im, (64,64), interpolation = cv2.INTER_AREA)
    # print("AFTER RESIZE", np.max(im), np.min(im))
    # im = torch.tensor(im)
    # print("AFTER TENSOR", torch.max(im), torch.min(im))
    # im = im.permute(2, 1, 0)
    # print("AFTER PERMUTE", torch.max(im), torch.min(im))
    
    # im = (im - 0.5)/0.5
    # print("AFTER NORMALIZATION", torch.max(im), torch.min(im))
    
    # # X.append(im)
    # # y.append(1)
    # print("AFTER", torch.max(im), torch.min(im))
    im = im.unsqueeze(0)
    # print(im.shape)
    
    X = torch.cat((X,im ), 0)
    y = torch.cat((y, torch.tensor(1).unsqueeze(0)), 0)
  if(count2>=3200):
    break 

print(count1, count2)

3200 3200


In [ ]:
.shape

torch.Size([6400])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train), torch.tensor(y_test) 

# X_train = X_train.permute(0, 3, 1, 2)

# X_test = X_test.permute(0, 3, 1, 2)

X_train.shape


my_dataset_train = TensorDataset(X_train, y_train) # create your datset
train_loader = DataLoader(my_dataset_train, batch_size=32, shuffle=True)

my_dataset_test = TensorDataset(X_test, y_test) # create your datset
test_loader = DataLoader(my_dataset_test, batch_size=32, shuffle=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model = Discriminator(ngpu=1).to(device)

In [ ]:
lr = 0.0005
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = GoogLeNet().to(device)
# model = Net().to(device)
batch_size = 64
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss_func = nn.CrossEntropyLoss()
loss_func = nn.BCELoss()

def train(epoch):
    model.train()
    correct =0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).float(), target.to(device).float()
        
        optimizer.zero_grad()
        #print(data.shape)
        output = model(data).view(-1)
        # print(data.shape)
        #print(target.shape)
        #print(output.shape)
        #print(output)
        loss = loss_func(output, target.float())
        loss.backward()
        optimizer.step()
        pred = torch.zeros(output.data.shape)
        pred[output.data >=0.5] = 1

        # pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        # if batch_idx % log_interval == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy{:.3f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item(), 100. * correct / len(train_loader.dataset) ))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device).float()
        target = target.to(device).long()
        output = model(data).view(-1)
        validation_loss = loss_func(output, target.float())
        # validation_loss += F.nll_loss(output, target, reduction="sum").item() # sum up batch loss
        # pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        pred = torch.zeros(output.data.shape)
        pred[output.data >=0.5] = 1
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(test_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    validation()
    model_file = 'DCGAN_discriminator_model_pretrained' + str(epoch) + '.pth'
    torch.save(model, model_file)
    print('\nSaved model to ' + model_file + '.')

Train Epoch: 1 [5408/5440 (99%)]	Loss: 0.137096	Accuracy83.309

Validation set: Average loss: 0.0002, Accuracy: 920/960 (96%)


Saved model to DCGAN_discriminator_model_1.pth.
Train Epoch: 2 [5408/5440 (99%)]	Loss: 0.000901	Accuracy98.603

Validation set: Average loss: 0.0000, Accuracy: 958/960 (100%)


Saved model to DCGAN_discriminator_model_2.pth.
Train Epoch: 3 [5408/5440 (99%)]	Loss: 0.009063	Accuracy99.485

Validation set: Average loss: 0.0000, Accuracy: 959/960 (100%)


Saved model to DCGAN_discriminator_model_3.pth.
Train Epoch: 4 [5408/5440 (99%)]	Loss: 0.049731	Accuracy99.540

Validation set: Average loss: 0.0001, Accuracy: 867/960 (90%)


Saved model to DCGAN_discriminator_model_4.pth.
Train Epoch: 5 [5408/5440 (99%)]	Loss: 0.000130	Accuracy99.449

Validation set: Average loss: 0.0000, Accuracy: 958/960 (100%)


Saved model to DCGAN_discriminator_model_5.pth.
Train Epoch: 6 [5408/5440 (99%)]	Loss: 0.000172	Accuracy99.779

Validation set: Average loss: 0.0000, Accuracy: 958/9

In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
pwd

'/content/drive/My Drive'

In [ ]:
ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)
PATH = "DCGAN_discriminator_model_6.pth"
model = torch.load(PATH)
model.eval()